# Introduction to `tcollections`

In [1]:
import sys
sys.path.append('../src')
import tcollections
from tcollections import tlist, group

## Collection Types

In [2]:
tl = tlist(range(10))
tl

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
tcollections.groupby(tl, lambda x: x % 2)

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

## Grouping Items
This section concerns grouping elements.

### Grouping Methods

To group elements of a collection according to a single key, you can use either `groupby` or `group.by`.

In [4]:
tcollections.group.by(tl, lambda x: x % 2)

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [5]:
tcollections.groupby(tl, lambda x: x % 2)

Groups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

Use `groupby_multi` or `group.multi` if you need nested groups.

In [6]:
tcollections.group.multi(tl, lambda x: (x % 2,))

NestedGroups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [7]:
tcollections.groupby_multi(tl, lambda x: (x % 2,))

NestedGroups({0: [0, 2, 4, 6, 8], 1: [1, 3, 5, 7, 9]})

In [8]:
tcollections.groupby_multi(tl, lambda x: (x % 2,x%3))

NestedGroups({0: NestedGroups({0: [0, 6], 2: [2, 8], 1: [4]}), 1: NestedGroups({1: [1, 7], 0: [3, 9], 2: [5]})})

### Aggregating Groups

In [9]:
tcollections.group.by(tl, lambda x: x % 2).agg(lambda x: sum(x))

{0: 20, 1: 25}

In [10]:
tcollections.group.multi(tl, lambda x: (x % 2,)).agg(lambda x: sum(x))

{0: 20, 1: 25}

In [11]:
tcollections.group.multi(tl, lambda x: (x % 2,x%3)).agg(lambda x: sum(x))

{0: {0: 6, 2: 10, 1: 4}, 1: {1: 8, 0: 12, 2: 5}}

In [12]:
tcollections.group.multi(tl, lambda x: (x % 2,x%3)).agg(lambda x: sum(x))

{0: {0: 6, 2: 10, 1: 4}, 1: {1: 8, 0: 12, 2: 5}}